# Momentum conversion

Momentum conversion in ERLabPy is exact with no small angle approximation, but is also very fast, thanks to the numba-accelerated trilinear interpolation in {mod}`erlab.analysis.interpolate`.

(nomenclature)=
## Nomenclature

Momentum conversion in ERLabPy follows the nomenclature from {cite:t}`ishida2018kconv`.

All experimental geometry can be classified into two configurations, Type 1 and Type 2, based on the relative position of the rotation axis and the analyzer slit. These can be further divided into 4 configurations depending on the use of photoelectron deflectors (DA).

Definition of angles differ for each geometry, but in all cases, $\delta$ is the azimuthal angle that indicates in-plane rotation, $\alpha$ is the angle detected by the analyzer, and $\beta$ is the angle along which mapping is performed.

For instance, imagine a typical Type 1 setup with a vertical slit that acquires maps by rotating about the `z` axis in the lab frame. In this case, the polar angle (rotation about `z`) is $\beta$, and the tilt angle becomes $\xi$.

The following table summarizes angle conventions for commonly encountered configurations.

```{eval-rst}
+---------------------------+---------------+-------------------+-----------+----------+-----------+-----------+-----------+
| Analyzer slit orientation | Mapping angle | Configuration     | Polar     | Tilt     | Deflector | Azimuth   | Analyzer  |
+===========================+===============+===================+===========+==========+===========+===========+===========+
| Vertical                  | Polar         | 1 (Type 1)        | ``beta``  | ``xi``   |           | ``delta`` | ``alpha`` |
+---------------------------+---------------+-------------------+-----------+----------+-----------+           |           |
| Horizontal                | Tilt          | 2 (Type 2)        | ``xi``    | ``beta`` |           |           |           |
+---------------------------+---------------+-------------------+-----------+----------+-----------+           |           |
| Vertical                  | Deflector     | 3 (Type 1 + DA)   | ``chi``   | ``xi``   | ``beta``  |           |           |
+---------------------------+               +-------------------+           |          |           |           |           |
| Horizontal                |               | 4 (Type 2 + DA)   |           |          |           |           |           |
+---------------------------+---------------+-------------------+-----------+----------+-----------+-----------+-----------+
```


:::{note}

Analyzers that can measure two-dimensional angular information simultaneously (e.g. time-of-flight analyzers) can be treated like hemispherical analyzers equipped with a deflector.

:::

In [ ]:
import erlab.plotting as eplt
import matplotlib.pyplot as plt

In [ ]:
import xarray
# Before starting, we set some options for xarray.

# - The first option, ``display_expand_data=False``, will collapse the data by
#   default in the `repr` of DataArrays and Datasets.

# - The second option, ``keep_attrs=True``, ensures that the attributes of the data
#   are kept when performing operations such as sum and mean. Since parameters for
#   momentum conversion such as angle offsets and inner potential are stored as
#   attributes, this is important.
%config InlineBackend.figure_formats = ["svg", "pdf"]
plt.rcParams["figure.dpi"] = 96


_ = xarray.set_options(display_expand_data=False, keep_attrs=True)

Let's generate some example data, this time in angle coordinates.

In [ ]:
from erlab.io.exampledata import generate_data_angles

dat = generate_data_angles(shape=(200, 60, 300), assign_attributes=True, seed=1).T
dat

## Converting to momentum space

Momentum conversion is done by the {meth}`convert <erlab.accessors.kspace.MomentumAccessor.convert>` method of the {meth}`DataArray.kspace <erlab.accessors.kspace.MomentumAccessor>` accessor. The bounds and resolution are automatically determined from the data if no input is provided. The method returns a new DataArray in momentum space.

:::{note}

For momentum conversion to work properly, the data must follow the conventions listed [here](data-conventions).

In [ ]:
dat_kconv = dat.kspace.convert()
dat_kconv

Let us plot the original and converted data side by side.

In [ ]:
fig, axs = plt.subplots(1, 2, layout="compressed")
eplt.plot_array(dat.sel(eV=-0.3, method="nearest"), ax=axs[0], aspect="equal")
eplt.plot_array(dat_kconv.sel(eV=-0.3, method="nearest"), ax=axs[1], aspect="equal")

## Setting parameters

Parameters that are needed for momentum conversion are the information about the experimental configuration, the inner potential $V_0$ (for photon energy dependent data), work function, and angle offsets. These parameters are all stored as data attributes. The `kspace` accessor provides various ways to access and modify these parameters.

See {meth}`configuration <erlab.accessors.kspace.MomentumAccessor.configuration>`, {meth}`inner_potential <erlab.accessors.kspace.MomentumAccessor.inner_potential>`, {meth}`work_function <erlab.accessors.kspace.MomentumAccessor.work_function>`, and {meth}`offsets <erlab.accessors.kspace.MomentumAccessor.offsets>` on how to access and modify each of these parameters.

First, let's check the angle offsets.

In [ ]:
dat.kspace.offsets

Since we haven't set any offsets, they are all zero. We will set the azimuthal angle to
60 degrees and the polar offset to 30 degrees and see what happens.

In [ ]:
dat.kspace.offsets.update(delta=60.0, beta=30.0)

In [ ]:
dat_kconv = dat.kspace.convert()
dat_kconv

Plotting the converted data again, we can see the effect of angle offsets on the conversion.

In [ ]:
fig, axs = plt.subplots(1, 2, layout="compressed")
eplt.plot_array(dat.qsel(eV=-0.3), ax=axs[0], aspect="equal")
eplt.plot_array(dat_kconv.qsel(eV=-0.3), ax=axs[1], aspect="equal")

## Converting coordinates only

Sometimes, we need to obtain the converted *coordinates* in momentum space without
modifying the data grid.

The code below demonstrates a possible use case where we convert the coordinates of a
cut to momentum space and overlay the location of the cut on the converted constant
energy map.

First, we select a cut from the original data along constant `beta`.

In [ ]:
cut = dat.qsel(beta=-10)
cut

In [ ]:
cut = cut.kspace.convert_coords()
cut

We can see that coordinate conversion adds momentum coordinates `kx` and `ky`, but does
not affect any existing coordinates. Now, let's annotate the cut location on the
constant energy map.

In [ ]:
fig, ax = plt.subplots()

dat_kconv.qsel(eV=-0.3).qplot(ax=ax, aspect="equal")

mdc = cut.qsel(eV=-0.3)
ax.plot(mdc.ky, mdc.kx, color="r")

## $k_z$-dependent data

Converting $k_z$-dependent data can be done in the exact same way by choosing an
appropriate value for the inner potential $V_0$. Let's generate some example data that
resembles photon energy dependent cuts.

In [ ]:
from erlab.io.exampledata import generate_hvdep_cuts

hvdep = generate_hvdep_cuts(seed=1)
hvdep

In this simulated data, the cuts are not through the BZ center, so the `beta` angle also
varies for each photon energy. 

We can convert this data to momentum space like before, after setting the inner
potential.

In [ ]:
hvdep.kspace.inner_potential = 10.0
hvdep_kconv = hvdep.kspace.convert()
hvdep_kconv

In [ ]:
hvdep.qsel(eV=-0.3).kspace.convert_coords().sel(hv=22)

In [ ]:
fig, axs = plt.subplots(1, 2, layout="constrained")
eplt.plot_array(hvdep.qsel(eV=-0.3).T, ax=axs[0])
eplt.plot_array(hvdep_kconv.qsel(eV=-0.3).T, ax=axs[1])

:::{note}

Since the generated example data is 2D-like, there is no visible periodicity in $k_z$, so it is impossible to estimate $V_0$. In practice, $V_0$ must be chosen so that the periodicity in $k_z$ matches the known periodicity of the lattice.

:::

### Annotating the photon energy

Each photon energy can be annotated on the converted data using {meth}`DataArray.kspace.convert_coords <erlab.accessors.kspace.MomentumAccessor.convert_coords>` with the data before conversion as described above. However, this only works for photon energies that exist in the data. 

The annotation can be done more easily by using {meth}`DataArray.kspace.hv_to_kz <erlab.accessors.kspace.MomentumAccessor.hv_to_kz>` on the converted data. The method returns the $k_z$ value for given photon energies based on the parameters stored in the data.

Here, we calculate the $k_z$ values for three different photon energies and select a
given binding energy.

In [ ]:
kz_values = hvdep_kconv.kspace.hv_to_kz([30, 45, 60]).qsel(eV=-0.3)
kz_values

We can now plot the calculated $k_z$ values on top of the converted data.

In [ ]:
fig, ax = plt.subplots(layout="constrained")

hvdep_kconv.qsel(eV=-0.3).T.qplot(ax=ax, aspect="equal")

for i in range(len(kz_values.hv)):
    kz = kz_values.isel(hv=i)

    ax.plot(kz.kx, kz, label=rf"$h\nu = {kz.hv:d}$ eV")

ax.legend()

Interactive conversion
----------------------

For three dimensional momentum conversion like maps or photon energy scans, an interactive window can be opened where you can adjust the parameters and see the effect right away.

There are three ways to invoke the GUI. The first one is to call {meth}`DataArray.kspace.interactive <erlab.accessors.kspace.MomentumAccessor.interactive>`:

```python
data.kspace.interactive()
```

The second option is to invoke the GUI directly with {func}`erlab.interactive.ktool`. If called with the second option, the name of the input data will be automatically detected and applied to the generated code that is copied to the clipboard.

```python
import erlab.interactive as eri

eri.ktool(data)
```

The final option is to trigger the GUI from the ImageTool with the "Open in ktool" menu in the View menu. The button will be disabled if the data is not compatible with {func}`erlab.interactive.ktool`.

The GUI is divided into two tabs.

```{image} ../images/ktool_1_light.png
:align: center
:alt: KspaceTool 1
:class: only-light
```

:::{only} format_html

```{image} ../images/ktool_1_dark.png
:align: center
:alt: KspaceTool 1
:class: only-dark
```
:::

The first tab is for setting momentum conversion parameters. The image is updated in real time as you change the parameters. Clicking the "Copy code" button will copy the code for conversion to the clipboard. The "Open in ImageTool" button performs a full three-dimensional conversion and opens the result in the ImageTool. 

```{image} ../images/ktool_2_light.png
:align: center
:alt: KspaceTool 2
:class: only-light
```
:::{only} format_html

```{image} ../images/ktool_2_dark.png
:align: center
:alt: KspaceTool 2
:class: only-dark
```
:::

The second tab provides visualization options. You can overlay Brillouin zones and high symmetry points on the result, adjust colors, apply binning, and more. The "Add Circle ROI" button allows you to add a circular region of interest to the image, which can be edited by dragging or right-clicking on it.

You can pass some parameters to customize the GUI. For example, you can set the Brillouin zone size/orientation and the colormap like this:

```python
data.kspace.interactive(
    avec=np.array([[-3.485, 6.03], [6.97, 0.0]]), rotate_bz=30.0, cmap="viridis"
)
```

See the documentation of {func}`erlab.interactive.ktool` for more information.